<img src="PillowLogo.jpg" width="400">

# Hands-On Lab: Home Price Estimation

## Introduction
In this lab, PACCAR is entering the real estate market and taking on Zillow! We will be estimating home prices, showing how good data and good science get us the best recommendations. Specifically, we'll walk through:
* A simple linear regression
* AutoML
* Data understanding & feature engineering

In [72]:
# Import any packages we will need to use for the analysis
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Import Data
When working in code, we first need to import our data. This can come from databases, spreadsheets, websites, etc.

```
[!] It is important to remember that data is loaded into your computer's memory. When people talk about "Big Data", it is commonly describing datasets that don't fit into memory!
```

In [6]:
# Read in our dataset and investigate the first several rows
home_data = pd.read_csv("../Data/Train.csv")
home_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


We should start by getting familiar with the dataset. Some common questions (and answers):

In [18]:
# What is our target variable?
print("SalePrice")
home_data.SalePrice.head(3)

SalePrice


0    208500
1    181500
2    223500
Name: SalePrice, dtype: int64

In [19]:
# How many rows of data do we have?
print('Number of rows: {}'.format(len(home_data)))

Number of rows: 1460


In [31]:
# How many attributes do we have for each row of data? What are the features?
print('Number of columns: {}'.format(home_data.shape[1]))
print(list(home_data))

Number of columns: 81
['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'

Thankfully, the city of Ames, Iowa does a terrific job of cataloging data. Have a look at *./Documentation/data_description.txt* for more information about what these inputs mean. Unfortunately, this is a rare treat but a state we should strive for.

In [135]:
# What data types are the features?
g = X_test.columns.to_series().groupby(X_test.dtypes).groups
print(g)

{dtype('int64'): Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object'), dtype('float64'): Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object'), dtype('O'): Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Bsm

## Prep Data for Modeling
When building predictive models, it is crucial that we split our data into *train* and *test* sets. This keeps our models honest as they are trained on one set of data, then tested on another. Later, we will talk about related concepts of overfitting and target leakage in relationship to these splits.

<img src="train_test_split.svg" width="600">

In [136]:
# Split our data columns into features and target tables
X = home_data.drop(columns=['SalePrice', 'Id'])
y = home_data[['SalePrice']]

In [137]:
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1984)

print("Training Rows: {} \nTest Rows: {}".format(len(X_train), len(X_test)))

Training Rows: 1022 
Test Rows: 438


## Linear Model
Our first approach will be a linear regression fit of home prices. This is similar to the type of fit we would perform in Minitab. Linear models are an excellent place to start due to their interpretability- we know exactly how predictions are being generated. 

On the other hand, linear models assume linear fits... an assumption that must be tested! Below we see an example of Anscombe's Quartet. From Wikipedia: 
*The data sets in the Anscombe's quartet are designed to have approximately the same linear regression line (as well as nearly identical means, standard deviations, and correlations) but are graphically very different. This illustrates the pitfalls of relying solely on a fitted model to understand the relationship between variables.*

Only the top-left quartet would be considered an appropriate model! 

<img src="AnscombesQuarter.png" width="600">

In [138]:
# One-hot encode any categorical variables
def one_hot_mixed_df(data):
    data_cat = data.select_dtypes(include='object')
    data_num = data.select_dtypes(exclude='object')
    ## One-hot transform the objects
    data_cat_dummies = pd.get_dummies(data_cat,drop_first=True)
    ## Join dummies with numerics
    data_lm = pd.concat([data_cat_dummies, data_num], axis=1, sort=False)
    return(data_lm)

# Apply transformations
X_lm = one_hot_mixed_df(X)

## Verify results
print('Total Columns: {}'.format(X.shape[1]))
print('Total Columns post-Dummy: {}'.format(X_lm.shape[1]))

Total Columns: 79
Total Columns post-Dummy: 245


In [139]:
print("Shape prior to NA cleanup: {}".format(X_lm.shape))

# NA values will crash the model fits, so let's see how to handle them
def count_NA(data):
    na_counts = data.isnull().sum(axis=0).reset_index(name='NACount')
    ## Print out the list of columns with missing values
    return(na_counts[na_counts.NACount>0])
    
print(count_NA(X_lm))

## We'll make a simplifying assumption that we can drop lot frontage and garage year built features. 
X_lm.drop(columns=['LotFrontage', 'GarageYrBlt'], inplace=True)

## For MasVnrArea, we will drop the NA rows. Note that rows also need to be dropped from our target!!!
row_drop_index =X_lm.MasVnrArea.notna()
X_lm = X_lm[row_drop_index]
y_lm = y[row_drop_index]

print("Shape of train after NA cleanup: {}".format(X_lm.shape))

Shape prior to NA cleanup: (1460, 245)
           index  NACount
210  LotFrontage      259
216   MasVnrArea        8
233  GarageYrBlt       81
Shape of train after NA cleanup: (1452, 243)


In [143]:
X_lm.select_dtypes(exclude='number').shape[1]

0

In [133]:
# Then split the rows into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1984)

Let's finally fit the linear model to an object called "reg", then look at some information about the model.

In [134]:
reg = LinearRegression().fit(X_train, y_train)                                                                                                  

ValueError: could not convert string to float: 'Partial'

In [126]:
print("Score:")
print(reg.score(X_test, y_test))
print("------------------------------------------------------------------")
print("Coefficients:")
print(reg.coef_)
print("------------------------------------------------------------------")
print("Intercepts:")
print(reg.intercept_)

Score:
0.9482388885416482
------------------------------------------------------------------
Coefficients:
[[ 4.29212061e+04  3.90909771e+04  4.29004391e+04  3.00295013e+04
   4.12193461e+04 -5.04633147e+03  3.72727299e+03  2.92418774e+03
   9.66278010e+02  5.46080766e+03 -1.35951603e+04  3.06668832e+03
   4.14858093e+03 -7.53778899e+03  1.10457324e+04 -2.64400682e+03
   4.51517926e+03 -3.13278607e+04  7.58036885e+03  1.00693568e+04
   1.18018160e+04 -7.23074765e+03 -2.32346576e+03  2.39504179e+04
  -6.78562032e+03 -5.88401390e+03  8.80548221e+03  4.68980785e+03
  -1.76261501e+04 -9.70465599e+03  1.57012653e+04 -8.79826437e+03
   2.66740840e+04  1.40028647e+04  3.61674037e+03  1.04096246e+04
  -3.09790861e+03  3.60134617e+03  8.24323343e+03  2.48471941e+04
  -9.49622974e+03  3.86107787e+03  6.39207795e+03  1.73463389e+04
   2.55494996e+03 -1.22613788e+03 -2.24281047e+03  1.21560683e+04
   3.84975981e+02  2.47745911e+03  4.05765800e+03  7.59347437e+03
   6.43765144e+04 -2.01697128e+04 -

So let's count the ways this was a bad model:
1. We didn't test any assumptions of normality
2. No checks of colinearity- can affect model performance
3. Are the numeric features meaningful? i.e. Just because MSSubClass is a number (60, 70, 80...), doesn't mean math operations on those numbers are valid.
4. We have a TON of columns, and usually want 100 rows per column. We're about 21x short.